<a href="https://colab.research.google.com/github/world970511/Py_SOLO_SchoolProject/blob/main/kinolights_TV_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**OTT 서비스 (TV) 분석**

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request

In [2]:
!pip install plotly

##**데이터 전처리**

In [3]:
#데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/world970511/elice_OTT/master/dataset/kinolight_all_data.csv",
                        filename="OTT.csv")
data = pd.read_csv('OTT.csv',encoding='utf-8')
tv_data=data.loc[data.kind == 'TV'] # kind가 tv인 것만 사용
tv_data.info

<bound method DataFrame.info of                                     title  ...          provider
0                  고요의 바다(The Silent Sea)  ...          ['넷플릭스']
2                           설강화(Snowdrop)  ...          ['디즈니+']
7             그 해 우리는(Our Beloved Summer)  ...   ['넷플릭스', '웨이브']
9           불가살(Bulgasal: Immortal Souls)  ...    ['넷플릭스', '티빙']
12              옷소매 붉은 끝동(The Red Sleeve)  ...  ['웨이브', '쿠팡플레이']
...                                   ...  ...               ...
27311  미니의 리본 가게 시즌 3(Minnie's Bow-Toons)  ...          ['디즈니+']
27312         만능 수리공 매니 시즌 1(Handy Manny)  ...          ['디즈니+']
27313         만능 수리공 매니 시즌 2(Handy Manny)  ...          ['디즈니+']
27314          아기를 부탁해! 토츠 시즌 2(T.O.T.S.)  ...          ['디즈니+']
27315            실험실의 비밀친구 시즌 3(Lab Rats)  ...          ['디즈니+']

[11428 rows x 10 columns]>

In [4]:
#중복제거 및 기타 제거
li=['genre','country','cast','director','provider',]

for i in li:
  tv_data[i]=tv_data[i].apply(lambda x:re.sub(r'\([^)]*\)', '',x.replace('[','').replace(']','').replace("'",'').replace("'",'')))
  tv_data[i]=tv_data[i].apply(lambda x:re.sub(r' ', '',x))
  tv_data[i]=tv_data[i].apply(lambda x: ','.join(set(x.strip().split(','))))

tv_data.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021.0,TV,15세이상관람가,"스릴러,SF",한국,"김선영,이무생,이성욱,허성태,이준,공유,배두나",최항용,NaN,넷플릭스
2,설강화(Snowdrop),2021.0,TV,15세이상관람가,드라마,한국,"장승조,유인나,정해인,김혜윤,정유진,지수,윤세아",조현탁,NaN,디즈니+
7,그 해 우리는(Our Beloved Summer),2021.0,TV,15세이상관람가,멜로/로맨스,한국,"조복래,최우식,노정의,김성철,김다미,박진주","김윤진,이단",NaN,"넷플릭스,웨이브"
9,불가살(Bulgasal: Immortal Souls),2021.0,TV,15세이상관람가,드라마,한국,"공승연,김우석,이진욱,박명신,이준,정진영,권나라",장영우,NaN,"넷플릭스,티빙"
12,옷소매 붉은 끝동(The Red Sleeve),2021.0,TV,15세이상관람가,드라마,한국,"박지영,이덕화,강훈,장혜진,장희진,준호,이세영",정지인,NaN,"쿠팡플레이,웨이브"


In [5]:
#불필요한 행 제거
tv_data= tv_data[tv_data['title']!='title']

#년도 정수형으로 변경
tv_data['year']=tv_data['year'].astype('float').astype('Int64')

#제작미정,시사/교양,기타(장르가 아니라 country로 가있음;;) 정리
# 제작미정인 경우 풀릴 예정인 곳과 아닌 곳이 있고 양이 너무 많아서 그냥 장르로 변경함
def r_ch(s,i):
  li=s.split(',')
  if i in li: 
    li.remove(i)
  return ','.join (li)


def add_ch(s,i):
  li=s.split(',')
  if i not in li: li+=[i]
  return ','.join (li)

li=['미정','시사/교양','기타']
for i in li:
    d=tv_data.loc[tv_data["country"].str.contains(i)]
    tv_data=tv_data.loc[tv_data["country"].str.contains(i)==False]

    d['country']=d['country'].apply(lambda x:r_ch(x,i))
    d['genre']=d['genre'].apply(lambda x:add_ch(x,i))

    tv_data=pd.concat([tv_data,d])

tv_data

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,"스릴러,SF",한국,"김선영,이무생,이성욱,허성태,이준,공유,배두나",최항용,NaN,넷플릭스
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,한국,"장승조,유인나,정해인,김혜윤,정유진,지수,윤세아",조현탁,NaN,디즈니+
7,그 해 우리는(Our Beloved Summer),2021,TV,15세이상관람가,멜로/로맨스,한국,"조복래,최우식,노정의,김성철,김다미,박진주","김윤진,이단",NaN,"넷플릭스,웨이브"
9,불가살(Bulgasal: Immortal Souls),2021,TV,15세이상관람가,드라마,한국,"공승연,김우석,이진욱,박명신,이준,정진영,권나라",장영우,NaN,"넷플릭스,티빙"
12,옷소매 붉은 끝동(The Red Sleeve),2021,TV,15세이상관람가,드라마,한국,"박지영,이덕화,강훈,장혜진,장희진,준호,이세영",정지인,NaN,"쿠팡플레이,웨이브"
...,...,...,...,...,...,...,...,...,...,...
23867,삼총사 시즌 3(The Musketeers Season 3),2016,TV,NaN,"드라마,어드벤처,기타",영국,,,NaN,웨이브
25022,JOB룡 이십끼(),2020,TV,NaN,"예능,기타",한국,유민상,,NaN,"웨이브,왓챠"
26598,빌로우 덱: 지중해의 요티 시즌 2(Below Deck Mediterranean),2017,TV,NaN,"예능,기타",미국,,,NaN,넷플릭스
23607,쉘위워크(Shall We Walk?),2017,TV,NaN,"예능,시사/교양,기타",한국,"이하늬,이제훈",,NaN,티빙


##**데이터셋**
- 키노라이츠(https://m.kinolights.com/) 12월 28일 기준 인기순으로 크롤링한 데이터(tv쇼만)
- title, year, kind, KMRB, genre, country, cast, director, runtime(min), provider 으로 구성
-provider: '쿠팡플레이', '왓챠', '라프텔', '아마존프라임비디오', '티빙', '웨이브', '넷플릭스', '네이버시리즈온', '디즈니+', 'AppleTV+', 'seezn'
- 총 11428개 데이터


In [6]:
tv_data.info()
#info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11428 entries, 0 to 23686
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         11428 non-null  object
 1   year          11410 non-null  Int64 
 2   kind          11428 non-null  object
 3   KMRB          10783 non-null  object
 4   genre         11428 non-null  object
 5   country       11428 non-null  object
 6   cast          11428 non-null  object
 7   director      11428 non-null  object
 8   runtime(min)  1084 non-null   object
 9   provider      11428 non-null  object
dtypes: Int64(1), object(9)
memory usage: 993.3+ KB


In [7]:
tv_data.tail()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
23867,삼총사 시즌 3(The Musketeers Season 3),2016,TV,NaN,"드라마,어드벤처,기타",영국,,,NaN,웨이브
25022,JOB룡 이십끼(),2020,TV,NaN,"예능,기타",한국,유민상,,NaN,"웨이브,왓챠"
26598,빌로우 덱: 지중해의 요티 시즌 2(Below Deck Mediterranean),2017,TV,NaN,"예능,기타",미국,,,NaN,넷플릭스
23607,쉘위워크(Shall We Walk?),2017,TV,NaN,"예능,시사/교양,기타",한국,"이하늬,이제훈",,NaN,티빙
23686,TV정보쇼 오!아시스(),2018,TV,NaN,"예능,시사/교양,기타",한국,"신현준,이지애",,NaN,티빙


In [8]:
#데이터에 적힌 provider 개수

provider_list=[]
li=tv_data['provider'].apply(lambda x: x.split(','))
for i in li:
    if i not in provider_list:
      provider_list+=i
provider_list=list(set(provider_list))
print(provider_list)

['티빙', 'seezn', '쿠팡플레이', 'AppleTV+', '네이버시리즈온', '넷플릭스', '디즈니+', '왓챠', '라프텔', '웨이브', '아마존프라임비디오']


##**분석**

###**각 플랫폼 당 tv콘텐츠 개수**

In [9]:
#독점 및 오리지널 표시

tv_data['provider']=tv_data['provider'].apply(lambda x: x.split(','))#provider 리스트 전환
tv_data.loc[:,'origin']=tv_data['provider'].apply(lambda x: True if len(x)==1 else False)#독점 여부
tv_data['provider']=tv_data['provider'].apply(lambda x: ','.join(x))#provider 다시 문자열 전환

In [10]:
tv_data.info

<bound method DataFrame.info of                                                title  year  ...   provider origin
0                             고요의 바다(The Silent Sea)  2021  ...       넷플릭스   True
2                                      설강화(Snowdrop)  2021  ...       디즈니+   True
7                        그 해 우리는(Our Beloved Summer)  2021  ...   넷플릭스,웨이브  False
9                      불가살(Bulgasal: Immortal Souls)  2021  ...    넷플릭스,티빙  False
12                         옷소매 붉은 끝동(The Red Sleeve)  2021  ...  쿠팡플레이,웨이브  False
...                                              ...   ...  ...        ...    ...
23867              삼총사 시즌 3(The Musketeers Season 3)  2016  ...        웨이브   True
25022                                     JOB룡 이십끼()  2020  ...     웨이브,왓챠  False
26598  빌로우 덱: 지중해의 요티 시즌 2(Below Deck Mediterranean)  2017  ...       넷플릭스   True
23607                           쉘위워크(Shall We Walk?)  2017  ...         티빙   True
23686                                  TV정보쇼 오!아시스()  2018  ...   

In [11]:
#개별 플랫폼 관련 데이터 전체 저장
for i in provider_list:
  tv_data.loc[tv_data["provider"].str.contains(i)].to_csv('/content/drive/MyDrive/project/엘리스 OTT 분석/data/'+i+'_all.csv',mode='w', index=False)

In [12]:
#시각화 라이브러리 호출
import plotly.express as px

#신규 데이터프레임
df= pd.DataFrame(columns=['provider','독점/오리지널','count','percent'])
for i in provider_list:
  d=tv_data.loc[tv_data["provider"].str.contains(i)]
  df.loc[len(df)]=[i,False,len(d[d['origin']==False]),len(d[d['origin']==False])*100/len(d)]
  df.loc[len(df)]=[i,True,len(d[d['origin']==True]),len(d[d['origin']==True])*100/len(d)]

#시각화 
fig = px.bar(df, x="provider", y="count",hover_data=['percent'], color="독점/오리지널", title="독점/오리지널 표기")
fig.show()

###**각 플랫폼 별 KMRB(관람가)**
- 전 연령대 관람가능 콘텐츠/성인용 콘텐츠의 양 파악

In [13]:
#KMRB가 NaN인 항목 제거
kmrb=tv_data
print('제거 이전:',len(kmrb))
kmrb = kmrb.dropna(subset = ['KMRB'], how = 'any', axis=0)
print('제거 이후:',len(kmrb))

제거 이전: 11428
제거 이후: 10783


In [14]:
#kmrb 리스트
kmrb_list=[]
li=kmrb['KMRB']
for i in li:
    if i not in kmrb_list:
      kmrb_list.append(i)
kmrb_list=list(set(kmrb_list))
print(kmrb_list)

['청소년관람불가', '전체관람가', '12세이상관람가', '15세이상관람가']


In [15]:
#시각화 라이브러리 호출
import plotly.express as px

#신규 데이터프레임
df2= pd.DataFrame(columns=['provider','관람가','count','percent'])

for i in provider_list:
  d=kmrb.loc[kmrb["provider"].str.contains(i)]
  for i2 in kmrb_list:
      df2.loc[len(df2)]=[i,i2,len(d[d['KMRB']==i2]),len(d[d['KMRB']==i2])*100/len(d)]

#시각화 
fig = px.bar(df2, x="provider", y="count",hover_data=['percent'], color="관람가", title="콘텐츠 관람가")
fig.show()

###**각 플랫폼 별 콘텐츠 제작 년도**
- 2021을 기준으로 최신 콘텐츠가 많은지 오래된 콘텐츠가 많은지 파악

In [16]:
#year가 NaN인 항목 제거
year_d=tv_data
print('제거 이전:',len(year_d))
year_d = year_d.dropna(subset = ['year'], how = 'any', axis=0)
print('제거 이후:',len(year_d))

제거 이전: 11428
제거 이후: 11410


In [17]:
#시각화 라이브러리 호출
import plotly.express as px

#신규 데이터프레임
df3= pd.DataFrame(columns=['provider','제작년도','count','percent'])

for i in provider_list:
  d=year_d.loc[year_d["provider"].str.contains(i)]
  df3.loc[len(df3)]=[i,'2021 이전',len(d[d['year']<2021]),len(d[d['year']<2021])*100/len(d)]
  df3.loc[len(df3)]=[i,'2021 이후',len(d[d['year']>=2021]),len(d[d['year']>=2021])*100/len(d)]

#시각화 
fig = px.bar(df3, x="provider", y="count",hover_data=['percent'], color="제작년도", title="근 1년 내 콘텐츠 비율")
fig.show()

###**플랫폼 별 콘텐츠 제작 국가**
- 해외/국내 콘텐츠 비율 파악

In [18]:
#country가 NaN인 항목 제거
country=tv_data
print('제거 이전:',len(country))
country = country.dropna(subset = ['country'], how = 'any', axis=0)
print('제거 이후:',len(country))

제거 이전: 11428
제거 이후: 11428


In [19]:
#데이터에 적힌 country 목록

country_list=[]
li=country['country'].apply(lambda x: x.split(','))
for i in li:
    if i not in country_list:
      country_list+=i
country_list=list(set(country_list))
country_list.remove('')
print(country_list)

['홍콩', '아이슬란드', '덴마크', '미국', '뉴질랜드', '네덜란드', '그리스', '아랍에미리트연합', '오스트리아', '콜롬비아', '사우디아라비아', '우크라이나', '핀란드', '헝가리', '터키', '일본', '칠레', '태국', '과테말라', '중국', '룩셈부르크', '체코', '루마니아', '필리핀', '스위스', '북한', '베트남', '캐나다', '러시아', '말레이시아', '이스라엘', '이탈리아', '브라질', '폴란드', '아르헨티나', '대만', '아일랜드', '노르웨이', '호주', '프랑스', '독일', '벨기에', '싱가포르', '멕시코', '한국', '남아프리카공화국', '스웨덴', '레바논', '포르투갈', '스페인', '크로아티아', '나이지리아', '인도', '세네갈', '요르단', '이집트', '키즈', '영국']


In [20]:
#국내/해외로 구분

#시각화 라이브러리 호출
import plotly.express as px

#신규 데이터프레임
df4= pd.DataFrame(columns=['provider','국내/해외','count','percent'])
for i in provider_list:
  d=tv_data.loc[tv_data['provider'].str.contains(i)]
  ko=len(d[d['country'].str.contains('한국')])
  n_ko=len(d[d['country'].str.contains('한국')==False])
  df4.loc[len(df4)]=[i,'국내',ko,ko*100/len(d)]
  df4.loc[len(df4)]=[i,'해외',n_ko,n_ko*100/len(d)]

#시각화 
fig = px.bar(df4, x="provider", y="count",hover_data=['percent'], color="국내/해외", title="국내/해외")
fig.show()

###**각 플랫폼 별 인기 장르**
- 플랫폼 별 인기 장르(전체)
- 영상매체 기준(애니메이션,다큐멘터리 등)
- 장르 기준(판타지,sf등)

In [21]:
#genre가 NaN인 항목 제거
genre=tv_data
print('제거 이전:',len(genre))
country = country.dropna(subset = ['genre'], how = 'any', axis=0)
print('제거 이후:',len(genre))

제거 이전: 11428
제거 이후: 11428


In [22]:
#데이터에 적힌 genre 목록
genre_list=[]
li=genre['genre'].apply(lambda x: x.split(','))
for i in li:
    if i not in genre_list:
      genre_list+=i
genre_list=list(set(genre_list))
genre_list.remove('')
print(genre_list)

['코미디', '범죄', '서스펜스', '전쟁', '시사/교양', '드라마', '예능', '미스터리', '가족', '다큐멘터리', '미정', '애니메이션', '서부극', '뮤지컬', '멜로/로맨스', 'SF', '음악', '공포', '판타지', '액션', '서사', '스릴러', '공연', '어드벤처', '기타']


In [23]:
#시각화 라이브러리 호출
import plotly.express as px

#신규 데이터프레임
df5= pd.DataFrame(columns=['provider','장르','count','percent'])
for i in provider_list:
  d=genre.loc[genre['provider'].str.contains(i)]
  for i2 in genre_list[1:]:
    g=len(d[d['genre'].str.contains(i2)])
    df5.loc[len(df5)]=[i,i2,g,g*100/len(d)]

#시각화 
fig = px.bar(df5, x="provider", y="count",hover_data=['percent'], color="장르", title="플랫폼 별 인기 장르(전체)")
fig.show()

In [24]:
#[애니메이션,다큐멘터리,미정,시사/교양,예능,공연,기타]을 제외한 장르별 비중

li=['애니메이션','다큐멘터리','미정','기타','시사/교양','예능','공연']
df5_2=df5.loc[df5['장르'].isin(li)==False]

"""각 플랫폼 별로 시각화=[아마존프라임비디오,디즈니+,웨이브,넷플릭스,쿠팡플레이,왓챠,seezn,티빙]만
라프텔은 99퍼가 애니메이션이라 제외
appletv와 네이버시리즈온은 데이터 양이 적어 제외시킴"""

#라이브러리 호출
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ama=df5_2.loc[df5_2['provider'].str.contains('아마존프라임비디오')]
di=df5_2.loc[df5_2['provider'].str.contains('디즈니+')]
wave=df5_2.loc[df5_2['provider'].str.contains('웨이브')]
net=df5_2.loc[df5_2['provider'].str.contains('넷플릭스')]
coo=df5_2.loc[df5_2['provider'].str.contains('쿠팡플레이')]
wat=df5_2.loc[df5_2['provider'].str.contains('왓챠')]
see=df5_2.loc[df5_2['provider'].str.contains('seezn')]
ti=df5_2.loc[df5_2['provider'].str.contains('티빙')]

# 서브플롯
specs = [[{'type':'domain'}, {'type':'domain'},{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=4, specs=specs)

fig.add_trace(go.Pie(labels=ama['장르'], values=ama['count'], name="아마존"),1, 1)
fig.add_trace(go.Pie(labels=di['장르'], values=di['count'], name="디즈니+"),1, 2)
fig.add_trace(go.Pie(labels=wave['장르'], values=wave['count'], name="웨이브"),1, 3)
fig.add_trace(go.Pie(labels=net['장르'], values=net['count'], name="넷플릭스"),1, 4)
fig.add_trace(go.Pie(labels=coo['장르'], values=coo['count'], name="쿠팡플레이"),2, 1)
fig.add_trace(go.Pie(labels=wat['장르'], values=wat['count'], name="왓챠"),2, 2)
fig.add_trace(go.Pie(labels=see['장르'], values=see['count'], name="seezn"),2, 3)
fig.add_trace(go.Pie(labels=ti['장르'], values=ti['count'], name="티빙"),2, 4)

#도넛모양으로 만들기
fig.update_traces(hole=.4, hoverinfo="label+percent+name",textinfo='none')

fig.update_layout(
    title_text='장르별 분류',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='아마존', x=0.09, y=0.8, font_size=11, showarrow=False),
                 dict(text='디즈니+', x=0.371, y=0.8, font_size=11, showarrow=False),
                 dict(text='웨이브', x=0.63, y=0.8, font_size=11, showarrow=False),
                 dict(text='넷플릭스', x=0.914, y=0.8, font_size=11, showarrow=False),
                 dict(text='쿠팡', x=0.09, y=0.19, font_size=11, showarrow=False),
                 dict(text='왓챠', x=0.371, y=0.19, font_size=11, showarrow=False),
                 dict(text='seezn', x=0.63, y=0.19, font_size=11, showarrow=False),
                 dict(text='티빙', x=0.914, y=0.19, font_size=11, showarrow=False)])
fig = go.Figure(fig)
fig.show()

In [27]:
#영상매체별[애니메이션,다큐멘터리,드라마,예능,공연] 분류

li=['애니메이션','다큐멘터리','예능','공연','드라마']
df5_3=df5.loc[df5['장르'].isin(li)]

"""각 플랫폼 별로 시각화=[아마존프라임비디오,디즈니+,웨이브,넷플릭스,쿠팡플레이,왓챠,seezn,티빙]만
라프텔은 99퍼가 애니메이션이라 제외
appletv와 네이버시리즈온은 데이터 양이 적어 제외시킴"""

#라이브러리 호출
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ama=df5_3.loc[df5_3['provider'].str.contains('아마존프라임비디오')]
di=df5_3.loc[df5_3['provider'].str.contains('디즈니+')]
wave=df5_3.loc[df5_3['provider'].str.contains('웨이브')]
net=df5_3.loc[df5_3['provider'].str.contains('넷플릭스')]
coo=df5_3.loc[df5_3['provider'].str.contains('쿠팡플레이')]
wat=df5_3.loc[df5_3['provider'].str.contains('왓챠')]
see=df5_3.loc[df5_3['provider'].str.contains('seezn')]
ti=df5_3.loc[df5_3['provider'].str.contains('티빙')]

# 서브플롯
specs = [[{'type':'domain'}, {'type':'domain'},{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=4, specs=specs)

fig.add_trace(go.Pie(labels=ama['장르'], values=ama['count'], name="아마존"),1, 1)
fig.add_trace(go.Pie(labels=di['장르'], values=di['count'], name="디즈니+"),1, 2)
fig.add_trace(go.Pie(labels=wave['장르'], values=wave['count'], name="웨이브"),1, 3)
fig.add_trace(go.Pie(labels=net['장르'], values=net['count'], name="넷플릭스"),1, 4)
fig.add_trace(go.Pie(labels=coo['장르'], values=coo['count'], name="쿠팡플레이"),2, 1)
fig.add_trace(go.Pie(labels=wat['장르'], values=wat['count'], name="왓챠"),2, 2)
fig.add_trace(go.Pie(labels=see['장르'], values=see['count'], name="seezn"),2, 3)
fig.add_trace(go.Pie(labels=ti['장르'], values=ti['count'], name="티빙"),2, 4)

#도넛모양으로 만들기
fig.update_traces(hole=.4, hoverinfo="label+percent+name",textinfo='none')

fig.update_layout(
    title_text='영상매체',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='아마존', x=0.09, y=0.8, font_size=11, showarrow=False),
                 dict(text='디즈니+', x=0.371, y=0.8, font_size=11, showarrow=False),
                 dict(text='웨이브', x=0.63, y=0.8, font_size=11, showarrow=False),
                 dict(text='넷플릭스', x=0.914, y=0.8, font_size=11, showarrow=False),
                 dict(text='쿠팡', x=0.09, y=0.19, font_size=11, showarrow=False),
                 dict(text='왓챠', x=0.371, y=0.19, font_size=11, showarrow=False),
                 dict(text='seezn', x=0.63, y=0.19, font_size=11, showarrow=False),
                 dict(text='티빙', x=0.914, y=0.19, font_size=11, showarrow=False)])
fig = go.Figure(fig)
fig.show()